# Importing libraries

In [1]:
import numpy as np
import time
import random
import pathlib
import threading
from queue import Queue
import copy
import math

import dlib
from deepface import DeepFace
import tensorflow
from torchvision.transforms import Resize, Lambda, Compose

import mediapipe as mp
import cv2

# Functions for face detection

This function compute the percentage of overlapping of two squares, applied to bounding boxes helps keeping track of a single face through different frames

In [2]:
def score_value(x,y,w,h,x1,y1,w1,h1):
    a=min([x+w-x1, x1+w1-x])
    b=min([y+h-y1, y1+h1-y])
    if (a>0) & (b>0):
        intersezione=a*b
        score=intersezione/(h*w+h1*w1-intersezione)
    else:
        score = 0
    return score

With the following function we extract from the 68 landmarks computed by dlib the 5 landmarks used for performing the alinement in the training set of sphereface model, in the 5 landmarks model are required two landmarks for center of the eyes, in the 68 landmark model are provided instead only the contour of the eyes, we compute the center of the eye using the mean of the leftmost and rightmost landmarks of each eye.

In [3]:
def landmarks_conversion(face_landmarks):
    right_eye=[(face_landmarks.part(36).x+face_landmarks.part(39).x)/2,(face_landmarks.part(36).y+face_landmarks.part(39).y)/2]
    left_eye=[(face_landmarks.part(42).x+face_landmarks.part(45).x)/2,(face_landmarks.part(42).y+face_landmarks.part(45).y)/2]
    nose=[face_landmarks.part(30).x, face_landmarks.part(30).y]
    mouth_right=[face_landmarks.part(48).x, face_landmarks.part(48).y]
    mouth_left=[face_landmarks.part(54).x, face_landmarks.part(54).y]
    landmarks=np.array([right_eye, left_eye, nose, mouth_right, mouth_left])
    return landmarks

Once we performed the landmark conversion we need a function that perform the alignement, this function gives as output the aligned foto of the correct size for sphereface model as well as an oversized one used by the game's scoreboard.

In [4]:
def alignment(foto, landmarks):
    crop_size = (96, 112)
    crop_size2 = (144, 144)
    a=int((crop_size2[0]-crop_size[0])/2)
    b=int((crop_size2[1]-crop_size[1])/2)

    reference = [ [30.2946, 51.6963],[65.5318, 51.5014],[48.0252, 71.7366],[33.5493, 92.3655],[62.7299, 92.2041] ]
    reference2=[]
    for lm in reference:
        reference2.append([lm[0]+a, lm[1]+b])

    landmarks_ = np.array(landmarks, dtype=np.float32)
    reference_ = np.array(reference2, dtype=np.float32)

    transformation_matrix, _ = cv2.estimateAffinePartial2D(landmarks_, reference_)

    foto_aligned_grossa = cv2.warpAffine(foto, transformation_matrix, crop_size2)
    foto_aligned = foto_aligned_grossa[b:b+crop_size[1], a:a+crop_size[0] ,:]

    return [foto_aligned, foto_aligned_grossa]

This function convert an image stored as a numpy array in a torch tensor where each value ranges from 0 to 1

In [5]:
trans = Compose([
    Lambda(lambda x: torch.from_numpy(x)),
    Lambda(lambda x: (x - 127.5 ) / 128.0), 
    Lambda(lambda x: x.permute(2,0,1))
])

# Models for face processing

Now we import the frontal face detector and the landmark position estimator from dlib

In [6]:
face_detector = dlib.get_frontal_face_detector()
dlib_facelandmark=dlib.shape_predictor("models/shape_predictor_68_face_landmarks.dat")

we also need to import sphereface model to extract features from faces

In [7]:
from net_sphere import sphere20a
model = sphere20a(feature=True)
model.load_state_dict(torch.load("models/sphere20a_20171020.pth"))

<IPython.core.display.Javascript object>

<All keys matched successfully>

# Auxiliary class definition

In this section we define two important classes: the first one represented an user and keep track of all the information related to the user, including a picture, the embeddings and the statistic of the game.

The second class instead represent a face and contain information of following observations of the same face in consecutive frames.

In [8]:
class Utente:
    def __init__(self, foto, colore, punti, lista_embeddings):
        self.foto = foto
        self.colore = colore #espresso come lista BGR
        self.punti=punti
        self.lista_embeddings=lista_embeddings
        self.aggiorna_statistiche()

    def aggiorna_statistiche(self):
        self.mean_=np.mean(self.lista_embeddings, axis=0)
        self.mean=self.mean_/np.linalg.norm(self.mean_)
        
class Faccia:
    def __init__(self, x, y, w, h, id_utente, lista_embeddings):
        self.x=x
        self.y=y
        self.w=w
        self.h=h
        
        self.id_utente=id_utente
        self.id_faccia=id_utente  #inizialmente id_faccia e id_utente corrispondono e sono una cosa negativa
        
        self.rilevamento=1
        self.reset_attesa()
        
        self.lista_embeddings=lista_embeddings
        
    def reset_attesa(self):
        self.attesa=30

# Main function for face detection and recognition

This is the more complex function used to perform face detection and user recognition, at first we run the frontal face detector, then for each face detected we compare it position to the faces detected in the previous frames, if there is exactly a face that was close enough we consider the new face as if it belong to the same user than the face detected before.

Then if we do not have enough data related to that face we run the model to estimate landmarks position, then we align the image and we compute multidimensional face embeddings.

Once we have computed the embedding we compute the mean of the embeddings related to that face (considering the ones collected in the previous frames) and we compute the cosine similiraty with the other user registered, if there is an user with cosine similarity higher than 0.7 we consider that face corresponding to the registered user.

If there is no such user that match the detected face we will contine to take embedding untill we have 10, if the face still do not correspond to any user we use that embeddings as material to register a new user.

At the end of the block there are also functions used to remove redundand faces (like faces that haven't be detected in a while)

In [9]:
def detect_and_process(frame, face_list, utenti_registrati, id_, cycle_counter, result_queue):
    
    gray=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces, scores, ids = face_detector.run(gray, 1, -1)
    
    for face in face_list:
        face.rilevamento=0
        
    #for every face detected
    for i in range(len(faces)):
        if (ids[i]==0) & (scores[i]>0):
            face_=faces[i]
                
            #computation of the bounding box
            x=face_.left()
            y=face_.top()
            w=face_.right()-face_.left()
            h=face_.bottom()-face_.top()

            #check if there are corresponding faces in the list of old frame's faces
            max_score=0
            best_match=0
            counter=0
            for i in range(len(face_list)): #this compute the closeness of each face respect to the old ones registered
                face=face_list[i]
                score=score_value(x, y, w, h, face.x, face.y, face.w, face.h)
                if score>0:
                    counter=counter+1
                if score>max_score:
                    max_score=score
                    best_match=i
            
            #if the face correspond to one and only one face in the old frames we consider as they are the same face
            sovrapposizione_minima=0.1
            if (max_score>sovrapposizione_minima) & (counter==1):
                face=face_list[best_match]
                face.rilevamento=1
                face.x=x
                face.y=y
                face.h=h
                face.w=w
                
                #if we do not have still identified the face we run the following code for face embeddings at every cycle, 
                #otherwise we run the following code once every 5 cycle
                if ((face.id_utente<0) | ((cycle_counter%5)==0)):
                    
                    #detection of face landmarks                
                    face_landmarks=dlib_facelandmark(gray, face_)                
                    landmarks=landmarks_conversion(face_landmarks)

                    #face alignement
                    foto_=alignment(frame, landmarks)
                    foto=foto_[1]

                    #computation of embeddings using sphereface model
                    emb = model(trans(foto_[0])[None])
                    emb_=emb.detach().numpy()[0]

                    #the frame embedding is added to the previous recorded ones
                    face.lista_embeddings=np.vstack([face.lista_embeddings, emb_])

                    #the belief on the user detected get updated, we check before if the last belief is confirmed and proceed checking
                    #other users if this is not the case
                    mean_face=np.mean(face.lista_embeddings, axis=0)
                    mean_face=mean_face/np.linalg.norm(mean_face)
                    v=0
                    if face.id_utente>=0:
                        user=utenti_registrati[face.id_utente]
                        mean=user.mean
                        v=np.dot(mean,mean_face)
                    if v<0.7:
                        old_belief=face.id_utente
                        face.id_utente=face.id_faccia
                        for i in range(len(utenti_registrati)):
                            if i!=old_belief:
                                user=utenti_registrati[i]
                                mean=user.mean                        
                                if np.dot(mean, mean_face)>0.7:
                                    face.id_utente=i

                    numero_dati=face.lista_embeddings.shape[0]
                    
                    #if we have reached a big number of data we use this data to create a new user if the face do not correspond
                    #to a registered user, otherwise we will use the data to update/add more data to the user
                    block_size=10
                    if (numero_dati%block_size)==0:
                        g=int(numero_dati/block_size)
                        #if this is a new user i register it
                        if face.id_utente<0:
                            a=random.randint(0, 225)+30
                            b=random.randint(0, 225)+30
                            c=random.randint(0, 225)+30
                            colore=[a,b,c]
                            punti=[0,0,0]
                            utenti_registrati.append(Utente(foto, colore, punti, face.lista_embeddings))
                        #if the user is already registered i update informations adding new embeddings
                        else:
                            samples=face.lista_embeddings
                            utenti_registrati[face.id_utente].lista_embeddings=np.vstack([face.lista_embeddings[(g-1)*block_size:,:], utenti_registrati[face.id_utente].lista_embeddings])
                            utenti_registrati[face.id_utente].aggiorna_statistiche()
                    
            #if no old face (entity) correspond to the current face (detected) we create an additional face (entity)
            else:
                #detection of face landmarks                
                face_landmarks=dlib_facelandmark(gray, face_)                
                landmarks=landmarks_conversion(face_landmarks)
                    
                #face alignement
                foto_=alignment(frame, landmarks)
                foto=foto_[1]

                #computation of embeddings using sphereface model
                emb = model(trans(foto_[0])[None])
                emb_=emb.detach().numpy()[0]
                
                face_list.append(Faccia(x,y,w,h,id_, emb_))
                id_=id_-1
                
                
            
        #update face entity informations
        for face in face_list:
            if face.rilevamento==0:
                face.attesa=face.attesa-1
                face.permanenza=0
            else:
                face.reset_attesa()
            
        #I subtract to the face list all the faces that hasn't been detected in a while
        new_face_list=[]
        for face in face_list:
            if face.attesa>0:
                new_face_list.append(face)

        #If there are multiple overlapping face entity we discard all the problematic faces
        proximity=np.zeros(len(new_face_list))
        for i in range(len(new_face_list)):
            face=new_face_list[i]
            for j in range(len(new_face_list)):
                face2=new_face_list[j]
                if i!=j:
                    score=score_value(face.x, face.y, face.w, face.h, face2.x, face2.y, face2.w, face2.h)
                    proximity[i]=max(score,proximity[i])
        comb=zip(proximity, new_face_list)
        face_list=[face for score, face in comb if score < 0.3]
        
        
    result_queue.put([face_list, utenti_registrati, id_])



# functions for gesture recognition

This block contains the two main functions that prepare the data (landmarks) obtained in the frame to be used in the gesture recognition model.

Distance_map returns the list of normalized reciprocal distances of every possible pair of landmarks in the hand.

Finger_angles returns the value of 10 angles selected among the landmarks. (Details in notebook RPS_Dataset.ipynb).

Finally, the function prep simultaneously applies these two functions to the landmarks of a hand.

In [10]:
#Returns the list of the all possible couples of the landmarks
def couples():
    a = [i for i in range(21)]
    b = []
    for i in a:
        for j in range(len(a)-(i+1)):
            b.append(([i, i+j+1]))
    return b

#Euclidean distance
def dist(p, q):
    distance = math.sqrt((p[0] - q[0])**2 + (p[1] - q[1])**2)
    return distance

def distance_map(landmark_list):
    couples_list = couples()
    distances = []
    
    #Compute distance for each possible couple
    for i in couples_list:
        p = [landmark_list.landmark[i[0]].x, landmark_list.landmark[i[0]].y] #Point 1
        q = [landmark_list.landmark[i[0]].x, landmark_list.landmark[i[1]].y] #Point 2
        distance = dist(p, q)
        distances.append(distance)

    #Normalization
    total_distance = sum(distances)/100
    distances = [distance / total_distance for distance in distances]  #Normalization step

    return distances

# importing models

importing the gesture recognition model trained by us

In [11]:
#importare modello gesture recognition
model_gesture = tensorflow.keras.models.load_model("models/RPSmodel.h5")
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

# additional functions used for gesture recognition

Hand_detection is the function that returns the landmarks (and the image with the landmarks drawn) given a frame. (Details in the notebook RPS_Dataset.ipynb).

Ok_detector returns the list of probabilities that each recognized hand in the frame is in the "ok" position, the pose used as the start game signal.

In [12]:
def hand_detection(photo):
    with mp_hands.Hands(min_detection_confidence=0.6, min_tracking_confidence=0.25, max_num_hands=4) as hands:

        image = cv2.cvtColor(photo, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # Detections
        results = hands.process(image)
        image.flags.writeable = True
        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #for drawing landmarks
        #if results.multi_hand_landmarks:  #if hand is detected
        #    for num, hand in enumerate(results.multi_hand_landmarks):
        #        mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS)

    return image, results

def ok_detector(hands): 
    likely_list = []
    for h in hands:
        d = np.asarray(distance_map(h))
        likely_list.append(model_gesture(d[None, :])[0][3])

    return likely_list


The gamestart function is used to determine if the game can start, meaning if both players have their thumbs up. In this function, the hand used by each player for playing is saved. This information will be important to make the final classification (rock, paper, or scissors) more robust. The hand used to start the game will be considered. This prevents the model from inadvertently classifying the other hand of the player, which may coincidentally be in a pose resembling those specific to the game.

In [13]:
def gamestart(dots, hand):
   
    indices = np.argsort(ok_detector(dots))[-2:]  #Index of the 2 most likely fists
            
    point = 0
    error = True
    while error:
        try:
            if(dots[indices[0]].landmark[point].x>dots[indices[1]].landmark[point].x):
                hand_pl1 = hand[indices[0]].classification[0].label 
                hand_pl2 = hand[indices[1]].classification[0].label
            else:
                hand_pl1 = hand[indices[1]].classification[0].label 
                hand_pl2 = hand[indices[0]].classification[0].label
            error = False
        except:
            point += 1

    #Return true if I detect 2 fists and if there is enough movement
    if np.sum(np.asarray(ok_detector(dots)) > 0.5) >= 2 and np.sum(np.asarray(ok_detector(dots)) > 0.5) >= 2:
        return True, hand_pl1, hand_pl2 
    
    return False, hand_pl1, hand_pl2 

This function returns the final classification of the hand poses of the players. As mentioned earlier, it only considers the hand used by each player to start the game. Additionally, it takes into account the positions of the hands in the scene to assign each hand to the correct player.

In [14]:
def RPS(dots, hands, hand_1, hand_2):
    left=[0]
    right=[0]
    
    point = 0
    error = True
    while error:
        try:
            mean_pos = np.mean([h.landmark[point].x for h in dots])  #Mean position of the hands

            for i in range(len(dots)):   #iteration for hands detected
                if dots[i].landmark[point].x < mean_pos: #Try to divide players
                    if hands[i].classification[0].label == hand_2:
                        d = np.asarray(distance_map(dots[i]))
                        right = model_gesture(d[None, :])
                else:
                    if hands[i].classification[0].label == hand_1:
                        d = np.asarray(distance_map(dots[i]))
                        left = model_gesture(d[None, :])
            error=False
        except:
            point += 1
            print(point)
    
    return np.argmax(left), np.argmax(right), min([np.max(left), np.max(right)])

This function returns who has won given the moves made by the players

In [15]:
def winner(a,b):
    if a == (b+1)%3: 
        return -1
    if a == b:
        return 0
    else:
        return 1

# importing user data and images

This section is used to import images needed for the game

In [16]:
s_image = cv2.imread("immagini\Scissor.jpeg")
p_image = cv2.imread("immagini\Paper.jpeg")
r_image = cv2.imread("immagini\Rock.jpeg")

winner_flag=cv2.imread("immagini\winner.png", cv2.IMREAD_UNCHANGED)
winner_flag=cv2.resize(winner_flag,(200,100))

image_list=[cv2.flip(r_image, 1), cv2.flip(p_image, 1), cv2.flip(s_image, 1)]

In [17]:
face_list=[]
utenti_registrati=[]

This part is used to import data of users registered in old games

In [18]:
for user in os.listdir('users_data/embeddings'):

  with open('users_data/embeddings/'+user, 'rb') as file:
    lista_embeddings = pickle.load(file)
  with open('users_data/colori/'+user, 'rb') as file:
    colore = pickle.load(file)
  with open('users_data/foto/'+user, 'rb') as file:
    foto = pickle.load(file)
  utenti_registrati.append(Utente(foto, colore, [0,0,0], lista_embeddings))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Main loop

For what concern the main function of face detection and recognition is handled using a thread this way the frame rate of the camera is not made slower by that.

The game is handled through a 4 state system, when the game is in the state 0 it wait for the signal to start the game (both player have to do the OK gesture), if the signal is recived the game switchs to state 1 where a countdown is displayed and at the end of that the simbols displayed by the users are classified. In the phase 2 scores are updated and then in phase 3 the result and the gesture detected are displayed on screen for 3 seconds, then the state automatically reset to 0.

The gesture identified is associated to a specific user according to the position in the screen, we assume that during the game only 2 players are present on screen and the rightmost hand is associated with the rightmost user.

In [22]:
dim1=80
dim2=20
sp=2
id_=-1
cycle_counter=0

classes = ["Rock", "Paper", "Scissors", "Ok"]
start = False
finish = False
ti = None 
to_be_reported = False
gamestate=0

start_time = time.time()
occupied = 0

result_queue = Queue()


cap = cv2.VideoCapture(0)
while cap.isOpened():
        
        #frame acquisition
        ret, frame = cap.read()
        frame2=copy.copy(frame)
        
        #The next block of code handle the thread for face detection and user identification procedures
        face_list_2=[]
        for face in face_list:
            face_list_2.append(copy.copy(face))  
        if occupied==0:
            processing_thread = threading.Thread(target=detect_and_process, args=(frame,face_list_2,utenti_registrati,id_,cycle_counter,result_queue))
            processing_thread.start()
            occupied=1  
        if result_queue.qsize()>0:
            new_data=result_queue.get()
            face_list=new_data[0]
            utenti_registrati=new_data[1]
            id_=new_data[2]
            occupied=0
            cycle_counter=cycle_counter+1
            
        #here we detect the order of players id in the image, this way we can associate each simbols to an user
        players=[]
        for face in face_list:
            if (face.rilevamento==1) and (face.id_utente>=0):
                players.append((face.x,face.id_utente))
        sorted_players=sorted(players, key=lambda x: x[0])

        
        #hand detection
        frame2, results = hand_detection(frame2)
        dots = results.multi_hand_landmarks
        hands = results.multi_handedness 
        
        #Game's state main routine
        #this part of the code is the main part that take care of game states
        text = " "
        if gamestate==0: #in state 0 we check if the users are doign the "start signal"
            if (dots is not None) and (len(dots)>1) and (len(players)==2):
                game_players=players.copy()
                start, hand_1, hand_2 = gamestart(dots, hands)
                if start:
                    ti = time.time()
                    gamestate+=1
        if gamestate==1: #in gamestate 1 we display the countdown and classify the simbols
            text=str(max(int(ti+3.9-time.time()),0))
            if ((time.time()-ti)>=5) and (dots is not None) and (len(dots)>1):
                out1, out2, confidence = RPS(dots, hands, hand_1, hand_2)
                to=time.time()
                #print(str(out1)+" "+str(out2)+" "+str(confidence))
                if (confidence>0.60) and (out1!=3) and (out2!=3):
                    gamestate+=1
                    to_be_reported=True
        if gamestate == 2: #in gamestate 2 scores are updated
            text=" "
            if to_be_reported:
                win = winner(out1, out2)
                if win==-1:
                    utenti_registrati[game_players[1][1]].punti[2]+=1
                    utenti_registrati[game_players[0][1]].punti[0]+=1
                    j=1
                elif win==0:
                    utenti_registrati[game_players[1][1]].punti[1]+=1
                    utenti_registrati[game_players[0][1]].punti[1]+=1
                    j=0
                else:
                    utenti_registrati[game_players[1][1]].punti[0]+=1
                    utenti_registrati[game_players[0][1]].punti[2]+=1
                    j=-1
                to_be_reported=False
                gamestate += 1
                ti=time.time()
        if gamestate==3 and (time.time() - ti <= 3): #in gamestate 3 results are displayed on screen
            if j!=0:
                for d1 in range(winner_flag.shape[0]):
                    for d2 in range(winner_flag.shape[1]):
                        if winner_flag[d1,200-d2-1,3]>0.5:
                            frame2[0+d1, 220+100*j+d2]=winner_flag[d1,200-d2-1,:3]
            frame2[65:165, 170:270] = cv2.resize(image_list[out2], (100,100))
            frame2[65:165, 370:470] = cv2.resize(image_list[out1], (100,100))  
        if gamestate==3 and (time.time() - ti >= 3): #here the game state is setted again to 0 to start a new game
            gamestate=0
        
        #Next blocks of code are related to the visual part of the game
        #display face bounding box
        for face in face_list:
            if face.rilevamento==1:
                if face.id_utente>=0:
                    colori=utenti_registrati[face.id_utente].colore
                else:
                    colori=[0,0,0]
                cv2.rectangle(frame2, (face.x,face.y), (face.x+face.w,face.y+face.h), (colori[0], colori[1], colori[2]), 2)
        frame2 = cv2.flip(frame2, 1)
        pad=np.ones((dim2, dim1+sp, 1))
        
        #update scoreboard
        for i in range(len(utenti_registrati)):
            try:
                colori=utenti_registrati[i].colore
                punti=utenti_registrati[i].punti
                
                cv2.rectangle(frame2, (sp,i*(dim1+dim2)+sp), (dim1-sp,i*(dim1+dim2)+dim1-sp), (colori[0], colori[1], colori[2]), 2*sp)
                frame2[i*(dim1+dim2)+dim1:(i+1)*(dim1+dim2),0:dim1+sp]=np.concatenate((pad*colori[0],pad*colori[1],pad*colori[2]),axis=2)
                frame2[i*(dim1+dim2)+2*sp:i*(dim1+dim2)+dim1-2*sp,2*sp:dim1-2*sp]=cv2.resize(utenti_registrati[i].foto, (dim1-4*sp,dim1-4*sp))
                testo = str(punti[0])+" "+str(punti[1])+" "+str(punti[2])
                posizione_testo = (15, (dim1+dim2)*i+dim1+12)
                font = cv2.FONT_HERSHEY_SIMPLEX
                scala_font = 0.5
                colore_font = (0, 0, 0)
                spessore_linea = 1
                cv2.putText(frame2, testo, posizione_testo, font, scala_font, colore_font, spessore_linea)
                
            except:
                pass
        
        #add text to the frame
        cv2.putText(frame2, text, (310, 70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 3)
        
            
        cv2.imshow('Window', frame2)
        
        key = cv2.waitKey(10)
        if key == ord('q'):
            break
            
cap.release()
cv2.destroyAllWindows()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Saving users data for next rounds

In [20]:
#saving users data
for i in range(len(utenti_registrati)):
    utente=utenti_registrati[i]
    numero_embeddings=utente.lista_embeddings.shape[0]
    numero_massimo=50
    with open('users_data/embeddings'+'/user_'+str(i), "wb") as file:
        if numero_embeddings<numero_massimo:
            pickle.dump(utente.lista_embeddings, file)
        else:
            r=random.sample(range(1, numero_embeddings), 50)
            pickle.dump(utente.lista_embeddings[r,:], file)
        
    with open('users_data/colori'+'/user_'+str(i), "wb") as file:
        pickle.dump(utente.colore, file)
    with open('users_data/foto'+'/user_'+str(i), "wb") as file:
        pickle.dump(utente.foto, file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>